In [1]:
import numpy as np
import matplotlib
import cv2

In [17]:
def display_image(window_name, img):
    """
    Displays image with given window name.
    :param window_name: name of the window
    :param img: image object to display
    """
    cv2.imshow(window_name, img.astype(np.uint8))
    cv2.waitKey(0)
    # cv2.destroyAllWindows()

def draw_landmarks(window_name, img, landmarks):
    img_ = np.copy(img)
    for landmark in landmarks:
        cv2.circle(img_, (int(landmark[0]), int(landmark[1])), 2, (0, 255, 0), 2)
    display_image(window_name, img_)

In [3]:
def l2_distance_transform_1D(edge_vector, positive_inf = np.inf, negative_inf = -np.inf):
    l = len(edge_vector)
    k = 0

    v = np.zeros(l, np.int32)

    z = np.empty(l + 1)
    z[0] = negative_inf
    z[1] = positive_inf

    dist_tf = np.zeros(l)

    q = 1
    while q < l:
        s = ((edge_vector[q] + q ** 2) - (edge_vector[v[k]] + v[k] ** 2)) / (2 * (q - v[k]))
        if s <= z[k]:
            k = k - 1
            continue
        else:
            k = k + 1
            v[k] = q
            z[k] = s
            z[k + 1] = positive_inf
            q = q + 1
    k = 0
    for q in range(l):
        while z[k + 1] < q:
            k = k + 1
        dist_tf[q] = (q - v[k]) ** 2 + edge_vector[v[k]]

    return dist_tf


def l2_distance_transform_2D(edge_function):
    dist_column_wise = np.zeros_like(edge_function, np.uint32)
    for col in range(edge_function.shape[1]):
        dist_column_wise[:, col] = l2_distance_transform_1D(edge_function[:, col])
    
    l2_dist_tf = np.zeros_like(edge_function, np.uint32)
    for row in range(edge_function.shape[0]):
        l2_dist_tf[row] = l2_distance_transform_1D(dist_column_wise[row])

    l2_dist_tf = l2_dist_tf * 255 / np.max(l2_dist_tf)
    return l2_dist_tf

In [4]:
img = cv2.imread("data/hand.jpg")

In [5]:
def compute_gradient(DistTF):
    kernel = np.array([[0, 0, 0], [-2, 0, 2], [0, 0, 0]])
    grad_x = cv2.filter2D(DistTF, -1, kernel)
    grad_y = cv2.filter2D(DistTF, -1, kernel.T)
    return grad_x, grad_y

In [6]:
edges = cv2.Canny(img, 125, 150)
BinEdge = np.where(edges == 255, 0, img.shape[0] ** 2 + img.shape[1] ** 2)

DistTF = l2_distance_transform_2D(BinEdge)

In [7]:
# Detect Gradients and Edges
Grad_x, Grad_y = compute_gradient(DistTF)
Grad_mag = np.sqrt(np.square(Grad_x) + np.square(Grad_y))


In [29]:
def compute_nearest_pts(landmarks, DistTF, grad_x, grad_y, grad_mag):
    y = landmarks[:, 1].astype(int)
    x = landmarks[:, 0].astype(int)
    nearest_pts = landmarks - np.multiply(np.divide(DistTF[y, x], grad_mag[y, x], where=grad_mag[y, x]!=0).reshape(-1, 1), np.dstack((grad_x[y, x], grad_y[y, x]))[0])
    return nearest_pts.astype(int)

def compute_affine_tf(source_pts, target_pts):
    source_pts_hom = np.hstack((source_pts, np.ones((len(source_pts), 1))))
    A = np.array([np.kron(np.eye(2), pt) for pt in source_pts_hom]).reshape(-1, 6)
    b = target_pts.reshape(-1, 1)

    x = np.linalg.pinv(A.astype(float)) @ b
    T = np.eye(3)
    T[:2, :] = x.reshape(2, 3)

    return T
    



In [31]:
landmarks_ip = np.loadtxt("data/hand_landmarks.txt", tuple, delimiter=',', converters={0: lambda x: int(str(x).strip('b\'(')), 1: lambda y: int(str(y).strip('b\')'))})
draw_landmarks('Shape Model ICP', img, landmarks_ip)

change = np.inf
T = np.eye(3)
landmarks_old = np.copy(landmarks_ip)
landmarks_new = np.zeros_like(landmarks_old)
while change > 1:
    landmarks_new = compute_nearest_pts(landmarks_old, DistTF, Grad_x, Grad_y, Grad_mag)
    T = T @ compute_affine_tf(landmarks_old, landmarks_new)
    change = np.sum(np.linalg.norm((landmarks_new - landmarks_old).astype(float), 2, 1))
    landmarks_old = landmarks_new

landmarks_final = T[:2, :2] @ landmarks_ip.T + T[:2, -1].reshape(-1, 1)
draw_landmarks('Shape Model ICP', img, landmarks_final.T)
cv2.destroyAllWindows()